# Node Toolbox

This chapter covers the core node types you will use in AI workflows. For each node, you will learn:

- What problem it solves
- The key parameters to understand
- What the output looks like
- One common mistake to avoid

---

## 1. Manual Trigger

**What it solves:** Starts the workflow when you click "Execute Workflow" in the editor.

### Key Parameters

| Parameter | Description |
|-----------|-------------|
| (none) | Manual Trigger has no configuration |

### Output

An empty item: `{ }`

This is normal. Triggers start the flow but do not produce data. The next node (usually Set/Edit Fields) creates the initial data.

### Common Mistake

**Expecting data from the trigger.** Manual Trigger outputs an empty object. Use Edit Fields to create your input data.

**Docs:** [Manual Trigger](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.manualtrigger/)

---

## 2. Edit Fields / Set

**What it solves:** Creates, renames, or modifies data fields. Use it to define input data or save intermediate results.

### Key Parameters

| Parameter | Description |
|-----------|-------------|
| **Name** | The field name to create or modify |
| **Value** | Fixed text or an expression like `{{ $json.text }}` |
| **Type** | String, Number, Boolean (usually String) |
| **Keep Only Set** | When ON, removes all other fields |

### Fixed vs Expression

| Mode | When to use |
|------|-------------|
| **Fixed** | Type a literal value (e.g., `AI in healthcare`) |
| **Expression** | Reference data from another node (e.g., `{{ $json.text }}`) |

### Output

Items with your fields added. Example:

- **Input:** `{ }`
- **Configuration:** Set `topic` = "AI in healthcare"
- **Output:** `{ "topic": "AI in healthcare" }`

### Common Mistake

**Accidentally enabling "Keep Only Set."** This removes all previous fields. Only enable it when you explicitly want to reset the data.

**Docs:** [Edit Fields (Set) Node](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.set/)

---

## 3. Chat Model Provider

**What it solves:** Provides an AI model (the "brain") for LLM Chain nodes to use. It does not process data directly.

### Available Providers

| Provider | Node Name | What it offers |
|----------|-----------|----------------|
| **OpenRouter** | OpenRouter Chat Model | Access to many models via one API |
| **OpenAI** | OpenAI Chat Model | GPT-4, GPT-3.5, etc. |
| **Google** | Google Gemini Chat Model | Gemini models |

### Key Parameters

| Parameter | Description |
|-----------|-------------|
| **Credential** | Your API key (stored in Settings → Credentials) |
| **Model** | Which model to use (e.g., `gpt-4`) |
| **Temperature** | Creativity: 0 = focused, 1 = creative (optional) |
| **Max Tokens** | Maximum response length (optional) |

### Connection Type

Chat Model nodes connect with a **dotted line** to LLM Chain nodes. This indicates a capability relationship, not data flow.

One Chat Model can serve multiple LLM Chain nodes.

### Output

None visible. The model is used by connected LLM Chain nodes.

### Common Mistake

**"No credentials" error.** Create a credential with your API key in Settings → Credentials before using this node.

**Docs:** [AI and LLM Nodes Overview](https://docs.n8n.io/integrations/builtin/cluster-nodes/)

---

## 4. Basic LLM Chain

**What it solves:** Sends a prompt to an AI model and returns the response. This is the core "AI call" in your workflow.

### Key Parameters

| Parameter | Description |
|-----------|-------------|
| **System Message** | Instructions defining the AI's role and rules |
| **Human Message** | The prompt — what you want the AI to do |
| **Chat Model** | Connection to a Chat Model provider (dotted line) |

### System Message vs Human Message

| Field | What goes here | Example |
|-------|----------------|--------|
| **System Message** | How the AI should behave | "You are a professional editor. Be concise." |
| **Human Message** | What you want done | "Summarize this: {{ $json.article }}" |

### Output

The AI response appears in a field called `text`:

```json
{ "text": "Here is the summary..." }
```

Use `{{ $json.text }}` in the next node to access this response.

### Common Mistake

**Forgetting to connect a Chat Model.** The node fails without a model. Check for the dotted line connection.

**Docs:** [Basic LLM Chain](https://docs.n8n.io/integrations/builtin/cluster-nodes/root-nodes/n8n-nodes-langchain.chainllm/)

---

## 5. Switch

**What it solves:** Routes data to different branches based on conditions. Think of it as if/else logic.

### Key Parameters

| Parameter | Description |
|-----------|-------------|
| **Value** | The field to check (e.g., `{{ $json.route }}`) |
| **Operation** | How to compare: Equals, Contains, etc. |
| **Rules** | List of conditions, each leading to an output branch |
| **Fallback Output** | Where data goes if no rules match |

### How Routing Works

1. An LLM classifies input and outputs a label (e.g., `refund`, `support`)
2. A Set node stores that label in a field called `route`
3. The Switch node checks `{{ $json.route }}` against rules:
   - If equals `refund` → Output 1
   - If equals `support` → Output 2
   - Fallback → Output 3

### Output

Data goes to **only one branch** — the first rule that matches. Other branches show empty output.

### Common Mistake

**Case sensitivity.** If the LLM outputs `Refund` but your rule checks for `refund`, it will not match.

**Docs:** [Switch Node](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.switch/)

---

## 6. Merge

**What it solves:** Combines data from multiple input branches into a single output.

### Key Parameters

| Parameter | Description |
|-----------|-------------|
| **Mode** | How to combine inputs (see table below) |

### Combine Modes

| Mode | What it does | When to use |
|------|-------------|-------------|
| **Combine by Position** | Merges item 1 from A with item 1 from B | Most AI workflows |
| **Append** | Puts all items into a single list | When you want separate items |
| **Keep Key Matches** | Joins items by a matching field | When items share an ID |

**Recommendation:** Start with "Combine by Position" for AI workflows.

### Output (Combine by Position)

If Branch A outputs: `{ "sentiment": "positive" }`  
And Branch B outputs: `{ "facts": ["fact1"] }`

Merged output:
```json
{ "sentiment": "positive", "facts": ["fact1"] }
```

### What Happens with Mismatched Item Counts

If Branch A has 3 items and Branch B has 2 items:
- Items 1 and 2 are merged
- Item 3 from Branch A is dropped

**Solution:** Ensure parallel branches produce the same number of items.

### Common Mistake

**Field name collisions.** If both branches have a field with the same name, one overwrites the other. Use Set nodes to rename fields before merging.

**Docs:** [Merge Node](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.merge/)

---

## 7. AI Agent (Preview)

**What it solves:** Lets an AI decide which tools to use and when. Unlike Basic LLM Chain (which runs once), the AI Agent can loop: call a tool, read the result, then decide to call another tool or finish.

### How It Differs from Basic LLM Chain

| Feature | Basic LLM Chain | AI Agent |
|---------|-----------------|----------|
| **Execution** | Single call | Loop until done |
| **Tools** | None | Can use HTTP, Code, Search, etc. |
| **Decision-making** | You control flow | Agent decides what to do |
| **Complexity** | Simple | More powerful, less predictable |

**When to use AI Agent:** When the task requires multiple steps that the AI should figure out on its own.

**When to use Basic LLM Chain:** When you know exactly what the AI should do (most workflows).

### Key Parameters

| Parameter | Description |
|-----------|-------------|
| **Agent Type** | How the agent reasons (Tools Agent is most common) |
| **System Message** | Instructions for the agent's behavior |
| **Chat Model** | Connection to a Chat Model provider (dotted line) |
| **Tools** | Nodes the agent can invoke (dotted line connections) |

### How the Loop Works

1. Agent receives your prompt
2. Agent decides: "I need more information" → calls a tool
3. Tool returns data
4. Agent decides: "Now I can answer" → outputs final response
5. (Or: "I need another tool" → back to step 2)

### Example: Search and Summarize

**Goal:** User asks a question, agent searches the web, then summarizes results.

**Setup:**
- AI Agent node with system message: "Answer questions by searching when needed. Summarize your findings."
- Connected tool: SerpAPI (web search) or HTTP Request node
- Connected model: OpenAI Chat Model

**What happens:**
1. User prompt: "What are the latest AI regulations in the EU?"
2. Agent thinks: "I need current information" → calls search tool
3. Search returns results
4. Agent thinks: "I have enough info" → writes summary
5. Output: `{ "output": "Based on my search, here is what I found..." }`

### Output

The agent's final response appears in the `output` field:

```json
{ "output": "Based on my search, here is what I found..." }
```

### Common Mistake

**Not defining tool boundaries.** Without clear instructions, agents may call tools unnecessarily or get stuck in loops. Be specific about when to use each tool.

**Docs:** [AI Agent Node](https://docs.n8n.io/integrations/builtin/cluster-nodes/root-nodes/n8n-nodes-langchain.agent/)

---

## Quick Reference: All Nodes

| Node | Purpose | Key Output |
|------|---------|------------|
| **Manual Trigger** | Start workflow manually | Empty `{ }` |
| **Edit Fields / Set** | Create or modify data | Your defined fields |
| **Chat Model Provider** | Provide AI model | None (capability) |
| **Basic LLM Chain** | Call AI, get response | `{ "text": "..." }` |
| **Switch** | Route based on conditions | Data in one branch |
| **Merge** | Combine parallel branches | Combined fields |
| **AI Agent** | AI with tools, loops until done | `{ "output": "..." }` |

---

## Next Steps

Now that you know the core nodes, proceed to the **Workflow Examples** chapter to see them in action with three common patterns.